> <p><small><small>This Notebook is made available subject to the licence and terms set out in the <a href = "http://www.github.com/google-deepmind/ai-foundations">AI Research Foundations Github README file</a>.

<img src="https://storage.googleapis.com/dm-educational/assets/ai_foundations/GDM-Labs-banner-image-C5-white-bg.png">

# Lab: Fine-Tune All The Parameters of Your SLM

<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_5/gdm_lab_5_3_fine_tune_all_parameters_of_your_slm.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

30 minutes

Teach a model that was trained on the next-token prediction task to generate responses to questions.

## Overview

In the last lab, you created a function that produces a response in the desired flashcard format. In this lab, you will now explore how you can train a language model to generate the responses for flashcards in a specific format. This helps to automate the generation of the answers for flashcards because the model automatically completes both the category and the response based on the questions you provide.

You will use two components to do this:

*   The transformer language model from the previous courses that can predict the next token.
*   A small new dataset with questions and answers, the Africa Galore QA dataset.

You will use the full-parameter fine-tuning method. That is, you will continue training your transformer on the new task by performing additional steps of gradient descent. In contrast to training a model from scratch, this will allow you to build a specialized model with fewer training iterations. The fine-tuned model will combine its language abilities and the other patterns it has learned during pre-training with the information about the task format.

This lab will guide you through the entire fine-tuning pipeline, which includes the following steps: Loading a pre-trained model, loading a new dataset, preparing the new dataset, fine-tuning the pre-trained model, and evaluation of the model.

### What you will learn:

By the end of this lab, you will be able to:

* Identify the similarities between fine-tuning and training from scratch.
* Recognize the similarities and differences between a general and a more specific task i.e., next-token prediction versus answering questions with a flashcard response.
* Prepare data for fine-tuning.
* Fine-tune an existing model.


### Tasks

**In this lab, you will**:
* Load a small dataset for a new task.
* Format the data to be compatible with a pre-trained model.
* Load a pre-trained model.
* Fine-tune the pre-trained model with the new data for the flashcard task.
* Evaluate the model.

If you are able to, we **highly recommend running the code in this lab on a Colab instance with a GPU**. See the section "How to use Google Colaboratory (Colab)" below for instructions on how to do this.



## How to use Google Colaboratory (Colab)

Google Colaboratory (also known as Google Colab) is a platform that allows you to run Python code in your browser. The code is written in cells that are executed on a remote server.

To run a cell, hover over a cell, and click the `run` button to its left. The run button is the circle with the triangle (▶). Alternatively, you can also click a cell and use the keyboard combination Ctrl+Return (or ⌘+Return if you are using a Mac).

To try this out, run the following cell. This should print today's day of the week below it.

In [ ]:
from datetime import datetime
print(f"Today is {datetime.today():%A}.")

Today is Thursday.


Note that the order in which you run the cells matters. When you are working through a lab, make sure to always run all cells in order. Otherwise, the code might not work. If you take a break while working on a lab, Colab may disconnect you; in that case, you have to execute all cells again before  continuing your work. To make this easier, you can select the cell you are currently working on and then choose __Runtime → Run before__  from the menu above (or use the keyboard combination Ctrl/⌘ + F8). This will re-execute all cells before the current one.

### Using Colab with a GPU

Follow these steps to run the activities in this lab on a GPU:

1.  In the top menu bar, click **Runtime**.
2.  Select **Change runtime type** from the dropdown menu.
3.  In the pop-up window under **Hardware Accelerator**, select **GPU** (usually listed as `T4 GPU`).
4.  Click **Save**.

Your Colab session will now restart with GPU access.

Note that access to GPUs is limited and at times, you may not be able to run this lab on a GPU. All activities will still work but they will run slower and you will have to wait longer for some of the cells to finish running.

## Imports



In this lab, you will fine-tune a transformer model. For this, you will use the Keras package for defining and training the transformer language model, the Pandas package, for loading the dataset, and the TensorFlow package for shuffling and padding the fine-tuning dataset.

Run the following cell to import the required packages.

In [ ]:
%%capture
# Install the custom package for this course.
!pip install "git+https://github.com/google-deepmind/ai-foundations.git@main"

import os # For setting Keras configuration variables.

# Provides configuration for Keras.
os.environ["KERAS_BACKEND"] = "jax"

from urllib.request import urlretrieve # For loading a model from a URL.

import keras # For defining and training the model.
import pandas as pd # For loading the dataset.
import tensorflow as tf # For shuffling the dataset.

# For displaying more readable error messages.
from IPython.display import display, HTML
from ai_foundations import training # For training your model.
from ai_foundations import generation # For prompting your model.
# For loading the tokenizer.
from ai_foundations.tokenization.bpe_tokenizer import BPEWordTokenizer
from ai_foundations import formatting # For formatting the training data.

keras.utils.set_random_seed(812) # For making the training reproducible.

## Load the tokenizer

-----
> **ℹ️ Tokenizer for fine-tuning:**
>
>As with any transformer language model, you need to tokenize the training data and prompts. When fine-tuning a model, it is crucial that you use the **same tokenizer** that was used when the model was originally trained.
>
>Consider the following. Assume that token 532 corresponds to "rice" in the pre-trained model. In this case, the original model would have likely learned a token representation that captures the information that "rice" is a noun, that it refers to something that can be eaten, etc. If during fine-tuning, you were to now map a different token to this ID, then all of that information would be overwritten and you would not be able to rely on the rich representations that the model learned during pre-training.
>
------

Run the following cell to load the tokenizer that was used for pre-training.

In [ ]:
url = "https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore_qa_tokenizer_3000.pkl"

tokenizer = BPEWordTokenizer.from_url(url)

## Load the pre-trained model

The following cell loads the pre-trained model. In Keras, this works by first defining a Keras model with the same hyperparameters as the original pre-trained model and then using the `model.load_weights` method that sets all weights of the model to the final parameters of the pre-trained model.

Run the following cell to download the model parameters and load the pre-trained model.

In [ ]:
model_url = "https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore_qa_1000ep.weights.h5"
model_filename = "africa_galore_qa_1000ep.weights.h5"
urlretrieve(model_url, model_filename)

# Define the model.
model = training.create_model(
    max_length=399,
    vocabulary_size=tokenizer.vocabulary_size,
)

model.load_weights(model_filename)

## Load and prepare the data

As a next step, you will load the Africa Galore QA dataset and apply the flashcard format you created in the previous lab.

Run the following cell to load the fine-tuning dataset and print the first entry.

In [ ]:
# Load the question-answer dataset.
africa_galore_qa = pd.read_json(
    "https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore_qa_v2.json"
)

africa_galore_qa.head(2)

### Format and tokenize

In this step, you will prepare the dataset for fine-tuning the model. As when you trained a model from scratch, you need to define the input and the output for training the model.   

This involves the following three steps:
*   Formatting the questions and answers.
*   Tokenizing them so that they can be used as input to a model.
*   Storing the tokenized questions and answers in lists that can be used as inputs and targets for fine-tuning the model.

Run the following cell to perform these three steps for each example in the Africa Galore QA dataset.



In [ ]:
encoded_questions = [] # List of formatted and tokenized questions.
encoded_answers = [] # List of formatted and tokenized answers.

# Iterate through each item in the dataset.
for idx, row in africa_galore_qa.iterrows():

    # Run the format_qa function from the previous lab to format the question
    # and the answer.
    question, answer = formatting.format_qa(row)

    # Tokenize the question and answer.
    tokenized_q = tokenizer.encode(question)
    tokenized_a = tokenizer.encode(answer)

    # Append the tokenized question to the list of all tokenized questions.
    encoded_questions.append(tokenized_q)
    encoded_answers.append(tokenized_a)

#### Coding Activity 1: Verify the encoded questions and answers

When preparing data, it is good practice to always verify that the examples have been correctly processed. This can be done, for example, by verifying that there are the same number of questions and answers in the `encoded_questions` and `encoded_answers` lists and by printing a sample of the processed outputs to manually inspect them.

------
> 💻 **Your task:**
>
> Verify that the questions and answers were encoded correctly.
>
> To do this:
> 1.   Print the first 10 tokens of the first two questions.
> 2.   Print the first 10 tokens of the first two answers.
> 3.   Reflect on the following questions:
>      * Why are the first tokens in the two questions the same?
>      * Why are even more tokens at the start of the answers the same?
>      * What is the length of the dataset?
>
------

In [ ]:
# Print the number of questions.
print(f"Number of questions: {len(encoded_questions)}")

# Raise an error if the length of the answers is not the same as the length of
# the questions.
assert(len(encoded_questions) == len(encoded_answers))

# Print the first 10 tokens of the first two questions.
print() # Add your code here.

# Print the first 10 tokens of the first two answers.
print() # Add your code here.

### Define input and target sequences

Recall that in course 01 Build Your Own Small Language Model you learned that language models are trained by defining a sequence of **input** tokens and a sequence of **target** tokens. The target tokens are a shifted version of the input tokens such that every element in the target is shifted by one position to the left. During training, the model then learns to predict the target token at every position $i$ from all input tokens up to position $i$. The parameters are updated according to the loss of each target token, so that during the next iteration of training, the model makes better predictions for each token in the training data.

When fine-tuning a model on a task such as flashcard generation, all of these considerations are still true. First, you also have to prepare the data as a sequence of input and target tokens. For question-answer tasks, such as the flashcard generation task, you therefore have to turn the question-answer pair into a single sequence of tokens. You can do this by concatenating the question and the answer:

`<start_of_turn>user What is Jollof rice?<end_of_turn><start_of_turn>model Category: Food. \nJollof rice is a...`

During fine-tuning, the model would then learn that after the sequence of tokens `["<end_of_turn>", "<start_of_turn>", "model"]`, it should predict "Category" followed by a category, a newline (`\n`), and an answer to the question.

In theory, you could therefore prepare the sequence of inputs and the sequence of targets for this example as follows (note: the exact sequence would depend on the tokenizer):

```python
inputs = ["<start_of_turn>", "user", "What", "is", "Jollof", "rice", "?", "<end_of_turn>", "<start_of_turn>", "model", "Category", ":", "Food", "\n", "Jollof", "rice", "is", ...]
targets = ["user", "What", "is", "Jollof", "rice", "?", "<end_of_turn>", "<start_of_turn>", "model", "Category", ":", "Food", "\n", "Jollof", "rice", "is", ...]
```

You could then continue training your model using these two sequences. This would likely work and your model would eventually become proficient in generating the flashcard responses in the correct format for a given question. Note, if you are training a model using these inputs and outputs, you are not only teaching it to provide effective answers but also how to ask questions. This is because the loss is not computed only for tokens in the response but also for the tokens in the prompt.

To avoid teaching the model unnecessary skills (recall that the task here is designed such that users always provide a question), you can limit the computation of the loss function to the tokens in the response. This means that the model still learns to base the response on the prompt, since it can attend to the tokens in the prompt when predicting tokens in the answer. In this case the weights are only updated such that the model performs better at the task of interest i.e., generating responses for the flashcard generator.

One technique you can use to ignore the tokens in the question from the loss computation is to replace them with the special `<PAD>` token in the target sequence. This is because the loss function is generally implemented such that it ignores the predictions at all positions where the target token is `<PAD>`. Applying this technique would then lead to the following pair of inputs and targets:

```python
# Inputs are the same as before.
inputs = ["<start_of_turn>", "user", "What", "is", "Jollof", "rice", "?", "<end_of_turn>", "<start_of_turn>", "model", "Category", ":", "Food", "\n", "Jollof", "rice", "is", ...]
targets = ["<PAD>", "<PAD>", "<PAD>", "<PAD>", "<PAD>", "<PAD>", "<PAD>", "<start_of_turn>", "model", "Category", ":", "Food", "\n", "Jollof", "rice", "is", ...]
```

This way, the model is only fine-tuned on the intended task of generating responses.

The following cell implements the preparation of the inputs and the targets using the encoded questions and answers from the previous cell. Note that `concatenated_masked` is set to a sequence where all tokens in the question are replaced with the special `<PAD>` token, which is retrieved from the tokenizer.

In [ ]:
encoded_inputs = []
encoded_targets = []
pad_token_id = tokenizer.pad_token_id
for idx in range(len(encoded_questions)):
    # Length of the question.
    len_q = len(encoded_questions[idx])
    # Concatenate a question and answer.
    concatenated_tokens = encoded_questions[idx] + encoded_answers[idx]
    # Concatenate q and a but with question masked with padding token.
    concatenated_masked = [pad_token_id] * len_q + encoded_answers[idx]
    # Add concatenated tokens to inputs, except the last token.
    encoded_inputs.append(concatenated_tokens[:-1])
    # Add concatenated tokens to outputs, shifted by 1 place.
    encoded_targets.append(concatenated_masked[1:])

### Other data preparation

As you have already encountered in the previous courses, you also need to pad and truncate individual examples so that you can group them into batches. Furthermore, shuffling the data can help with more efficient training as it can avoid that very similar examples appear in the same batch during training.

Note that when fine-tuning, you have to adhere to the maximum length of the pre-trained model. Rather than inferring the maximum sequence length from the data, you are therefore extracting the maximum sequence length from the pre-trained model.

<br>

------
> **ℹ️ Info: Batch size during fine-tuning**
>
>The batch size for fine-tuning is often set to a smaller value than during pre-training. This is in part due to memory constraints, as all examples in a batch need to be stored on a GPU. Additionally, this often leads to better model performance. This is because the gradient estimates from small batches tend to be noisier than from large batches, and this noise can act as a form of regularization that makes it less likely that the model overfits.
>
-------

<br>

Run the following cell to perform padding, truncating, and batching and shuffling.

In [ ]:
# Maximum length for an input text. input_shape[0] is the batch size.
max_input_length = model.input_shape[1]
max_output_length = model.output_shape[1]

# The input sequences are the padded questions.
# Use Keras for padding and truncating.
input_sequences = keras.preprocessing.sequence.pad_sequences(
    encoded_inputs,           # Data to be truncated.
    maxlen=max_input_length,  # Maximum length.
    padding="post",           # Pad at the end if necessary
    truncating="post",        # Truncate at end if necessary.
    value=tokenizer.pad_token_id # The ID of the pad token.
)

# The target sequences are the padded answers.
target_sequences = keras.preprocessing.sequence.pad_sequences(
    encoded_targets,
    maxlen=max_output_length,
    padding="post",
    truncating="post",
    value=tokenizer.pad_token_id
)

# Create TensorFlow dataset to prepare sequences.
tf_dataset = tf.data.Dataset.from_tensor_slices(
    (input_sequences, target_sequences)
)

# Randomly shuffle the dataset.
tf_dataset = tf_dataset.shuffle(buffer_size=len(input_sequences))

# Specify batch size.
batch_size = 4

# Create batches.
batches = tf_dataset.batch(batch_size)

## Fine-tuning the model

With your data being prepared, you can now fine-tune your model on the flashcard generation task. For fine-tuning a Keras model that implements a transformer, you can continue the original training process.

The way to do this is by defining a model and loading its parameters (as you did at the beginning of this lab) and then continuing training with the  [`model.fit()`](https://keras.io/api/models/model_training_apis/)  method that you have been using to train models from scratch.

### Learning rate

One important consideration when fine-tuning a model is the learning rate. Recall from the course 03 Design and Train Neural Networks that the learning rate determines how much the weights are updated during each step of stochastic gradient descent.

Importantly, when fine-tuning a model, you want to retain the knowledge that has already been learned rather than making significant updates. As such, you generally want to use a smaller learning rate during fine-tuning than during pre-training. A common learning rate for pre-training is 0.0001 (1e-4) and for fine-tuning, you generally want to use a learning rate that is one or two orders of magnitude lower, so a value between 0.00001 (1e-5) and 0.000001 (1e-6) tends to be well-suited. If you choose higher values, then the learning process may end up being more susceptible to the problem of **catastrophic forgetting**.

<br />

------
> **ℹ️ Info: Catastrophic forgetting**
>
> When fine-tuning a model, it initially has all the abilities that it gained from the previous dataset. During fine-tuning, you train it only on new, additional data. If you fit the model parameters too closely to the patterns in the fine-tuning dataset, the model may "forget" what it has learned during pre-training, an issue often referred to as **catastrophic** forgetting. This is another reason why it is important to find the proper point to stop training on the new data. You want to make sure that you stop training after the model has adjusted to the new task, but before the knowledge gained during pre-training is overwritten.
------

<br />


Set the learning rate to 0.00005 in the following cell below and run the cell to update the learning rate of the optimizer.

In [ ]:
# @title Set the learning rate
learning_rate = 5e-5 # @param {type: "number"}
model.optimizer.learning_rate = learning_rate

### Number of epochs

The number of epochs that are needed for successful training strongly depends on the size of the dataset and the learning rate. The smaller the learning rate, the more training updates you will need. Likewise, if your dataset was ten times as big, one epoch of the big dataset would equal ten epochs of the small dataset. This would make it more diverse but also more expensive. Remember, if you train for too long, the model will experience catastrophic forgetting or another form of overfitting.

Run the following cell to set the number of epochs. We recommend you start with a value of 50 and increase it only if the model does not appear to have learned the task format after fine-tuning.

In [ ]:
num_epochs = 50 # @param {type: "number"}

### Callback function

When you trained the model in the course 01 Build Your Own Small Language Model, you implemented a callback function. This function allows you to monitor the progress of training and get a sense of what the model has learned after each epoch. This enables you to determine when to stop training and whether the setting of the learning rate is reasonable.

Run the following code to define the callback function. Note, since you are interested in how well the model does on the flashcard generation task, you should make sure that the prompt is similar to the prompts in the fine-tuning dataset.

In [ ]:
prompt = "<start_of_turn>user\nWhat is Jollof rice?<end_of_turn>\n"

prompt_ids = tokenizer.encode(prompt)
text_gen_callback = training.TextGenerator(
    max_tokens=30,
    start_tokens=prompt_ids,
    tokenizer=tokenizer,
)

### Coding Activity 2: Train the model



------
> 💻 **Your task:**
>
> Fine-tune the model by calling the `model.fit()` function. Make sure that you
> set the following parameters:
> * `x`: This should be set to the fine-tuning batches stored in `batches`.
> * `epochs`: This should be set to the number of epochs.
> * `callbacks`: Set this to a list that contains `text_gen_callback` to make sure the model executes the callback function after each epoch.
>
> After you have added the call to `model.fit()` below, run the cell to fine-tune the model and observe the training progress.
------

In [ ]:
# Add your code here.

### Activity 3: Test the model

In the first lab of this course, you tested how the pre-trained model responded to questions. Now that you have fine-tuned your model, repeat this exercise.



------
> 💻 **Your task:**
>
> Run the code with different statements and questions. Enter the texts in the prompt field and run the model. When you check the box `is_question`, the prompt is formatted using the turn-taking delimiters. Tick this checkbox for questions but run statements without adding the delimiters.
>
> Generate responses for the following statements and questions:
> *   Jollof rice is
> *   What is Jollof Rice?
> *   What is Tagine?
> *   What is Mount Kilimanjaro?
> *   What is Mount Aconcagua?
>
> Consider the following questions:
>
> *   How does the fine-tuned model perform for the statements?
> *   How does it perform for the questions?
> *   Do you see a difference in the responses between the question about the African mountain (Mount Kilimanjaro) and the question about the South American mountain (Mount Aconcagua)?
>
------

In [ ]:
# @title Prompt your fine-tuned model
question = "Jollof rice is" #@param {type: "string"}
is_question = True #@param {type: "boolean"}
if is_question:
    prompt = "<start_of_turn>user\n" + question + "<end_of_turn>\n"
else:
    prompt = question
num_tokens_to_generate = 30 #@param {type: "number"}
generated_text, probs = generation.generate_text(
    prompt,
    num_tokens_to_generate,
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    sampling_mode="greedy" # To generate the highest probability generation.
)

print("Generated text: ", fill(generated_text, replace_whitespace=False))
print("\n")

### What did you observe?

You likely observed that the model can still generate useful continuations for prompts that contain the beginning of a statement such as "Jollof rice is". Unlike in the first lab of this course however, you likely observed that the model can generate effective responses to questions, and that they now also follow the special flashcard format.

You likely also noticed that the question about the South American mountain, Mount Aconcagua, did not yield a sensible response. This is because neither the pre-training data of the model (the Africa Galore dataset) nor the fine-tuning data contains any information on this mountain.

## Summary

In this lab, you fine-tuned a transformer for generating responses for an automatic flashcard generator. This turned a language model that was only optimized for autocompletion of statements into a model that can respond to user questions.

This process involved several steps:

- **Formatting a new dataset:** You formatted the Africa Galore QA dataset by adding the special delimiters `"<start_of_turn>"` and `"<end_of_turn>"`, and combined category and response to create a flashcard-style format.

- **Loading a pre-trained model:** You loaded a model that already had abilities to process text. In particular, it was already able to predict the next word for a given context but still lacked the ability to generate answers to questions and lacked the ability to generate responses in the specific flashcard format.

- **Preparing the dataset:** You turned the question-answer pairs into a format that can be used to continue training a language model that has been optimized to generate texts by predicting one token at a time.

- **Fine-tuning the model:** You continued training the model using a question-answer dataset.

This lab also walked you through the important hyperparameters to consider when fine-tuning a model: the **learning rate**, the **number of epochs**, and the **batch size**.


In the next activities, you will learn about the idea of foundation models and
explore how you can achieve better results by using a larger model that has been pre-trained on a lot more data. This will allow you to build a model that can generate answers for questions on more diverse topics, such as the question about the South American mountain Mount Aconcagua.

## Solutions

The following cells provide reference solutions to the coding activities in this notebook. If you really get stuck after trying to solve the activities yourself, you may want to consult these solutions.

It is recommended that you *only* look at the solutions after you have tried to solve the activities *multiple times*. The best way to learn challenging concepts in computer science and artificial intelligence is to debug your code piece-by-piece until it works, rather than copying existing solutions.


If you feel stuck, you may want to first try to debug your code. For example, by adding additional print statements to see what your code is doing at every step. This will provide you with a much deeper understanding of the code and the materials. It will also provide you with practice on how to solve challenging coding problems beyond this course.


To view the solutions for an activity, click the arrow to the left of the activity name. If you consult the solutions, do not copy and paste them into the cells above. Instead, look at them, and type them manually into the cell. This will help you understand where you went wrong.


### Coding Activity 1

In [ ]:
print(encoded_questions[0][:20])  # Print first 20 tokens of first question.
print(encoded_questions[1][:20]) # Print first 20 tokens of second question.
print(encoded_answers[0][:30]) # Print first 30 tokens of first answer.
print(encoded_answers[1][:30]) # Print first 30 tokens of second answer.

### Coding Activity 2

In [ ]:
# Add this code to the cell above for fine-tuning the model.
history = model.fit(
    x=batches,
    epochs=num_epochs,
    callbacks=[text_gen_callback]
)